## Chapter 14: Deploying to the Cloud – Azure Container Apps

In the previous chapter, you learned how to deploy your Aspire application to a Kubernetes cluster using the `aspirate` tool. While Kubernetes offers flexibility and control, it also comes with operational complexity. For many teams, a **platform‑as‑a‑service (PaaS)** or **serverless containers** solution is more appealing. Azure Container Apps (ACA) is a serverless container platform that abstracts away the Kubernetes control plane while still providing powerful features like automatic scaling, ingress, and Dapr integration.

In this chapter, you’ll learn:

- Why Azure Container Apps is an ideal production target for .NET Aspire.
- How to use the Azure Developer CLI (AZD) to provision infrastructure and deploy your application.
- How Aspire integrates with AZD to generate ACA resources automatically.
- How to configure secrets, environment variables, and scaling rules.
- How to monitor your deployed application using Azure Monitor and Application Insights.

By the end, you’ll be able to take your e‑commerce application and deploy it to the cloud with minimal manual steps.

---

### 14.1 Why Azure Container Apps?

Azure Container Apps (ACA) is a fully managed serverless container service that runs on top of Kubernetes but hides the complexity. It offers:

- **Serverless scaling**: Scale to zero when idle, and scale out based on HTTP traffic, events (KEDA), or CPU/memory.
- **Built‑in ingress**: HTTP and HTTPS with automatic TLS termination.
- **Service discovery** via internal DNS names.
- **Dapr integration** for microservices patterns (state, pub/sub, bindings).
- **Managed identity** and integration with Azure Key Vault.
- **Logging and monitoring** via Azure Monitor and Log Analytics.

ACA is a perfect match for Aspire applications because it aligns with the cloud‑native principles Aspire promotes. Moreover, the Azure Developer CLI (AZD) provides a streamlined workflow that understands Aspire manifests, making deployment almost automatic.

---

### 14.2 Prerequisites

Before we begin, ensure you have:

- An **Azure subscription** (free trial works).
- The **Azure Developer CLI** installed. Install it via:

  ```bash
  powershell -ex AllSigned -c "Invoke-RestMethod 'https://aka.ms/install-azd.ps1' | Invoke-Expression"  # Windows
  curl -fsSL https://aka.ms/install-azd.sh | bash  # Linux/macOS
  ```

- The **.NET Aspire workload** and your application code from previous chapters.
- **Docker** running (AZD will build container images).

Optionally, install the **Azure CLI** and log in (`az login`) for additional management, but AZD handles authentication.

---

### 14.3 Preparing the Aspire App for AZD

Aspire templates include built‑in support for AZD. When you create a new project with `aspire-starter`, it includes a folder structure ready for AZD. However, our application has evolved, so we need to ensure it’s ready.

The key requirement is that the AppHost can generate a manifest, and AZD will use that manifest to understand what resources to create in Azure. Specifically, AZD looks for a `azure.yaml` file in the root of your repository that tells it how to build and deploy each service.

#### 14.3.1 Adding an `azure.yaml` File

Create a file named `azure.yaml` in the root of your solution (where the `.sln` file is). This file defines the services in your application and how to build them.

```yaml
# azure.yaml
name: myaspireapp
services:
  apiservice:
    project: ./MyAspireApp.ApiService/MyAspireApp.ApiService.csproj
    language: dotnet
    host: containerapp
  webfrontend:
    project: ./MyAspireApp.Web/MyAspireApp.Web.csproj
    language: dotnet
    host: containerapp
  workerservice:
    project: ./MyAspireApp.WorkerService/MyAspireApp.WorkerService.csproj
    language: dotnet
    host: containerapp
```

If you have additional services (like a worker), list them here. The `host: containerapp` tells AZD to deploy them as Azure Container Apps.

#### 14.3.2 The AppHost Manifest for AZD

When you run `azd up`, AZD will execute your AppHost in manifest mode (similar to how we did manually) to generate the resource graph. It then translates that graph into Azure Container Apps resources. Therefore, your AppHost must be able to run without errors (all resources properly defined). Ensure that any parameters are either provided via environment variables or you have default values for local manifest generation.

For production, you'll want to use real Azure services (like Azure Database for PostgreSQL) instead of containers. You can model these in the AppHost using the Azure hosting packages (e.g., `AddAzurePostgresFlexibleServer`). This tells AZD to provision those services in Azure. For simplicity, we'll keep using the container versions, but in production you'd replace them with managed services.

---

### 14.4 Initializing the AZD Environment

Open a terminal at the solution root and run:

```bash
azd init
```

You'll be prompted:

- **Environment name**: e.g., `myapp-dev`.
- **Subscription**: Select your Azure subscription.
- **Location**: Choose a region (e.g., `eastus`).

AZD will create an `azure` directory with configuration files, including `main.bicep` (infrastructure as code) and `main.parameters.json` for parameters.

It also creates a `.azure` folder for environment‑specific settings.

---

### 14.5 Understanding the Generated Bicep

AZD generates a `main.bicep` file that defines the Azure resources needed:

- **Resource group** (if not existing).
- **Log Analytics workspace** for monitoring.
- **Container Apps environment** (the ACA environment).
- **Container Apps** for each of your services.
- **Azure Container Registry** (ACR) to store images.
- Possibly other resources like PostgreSQL, if you modeled them.

You can customize this file to add more resources (like a database) or change configurations. However, AZD also integrates with the Aspire manifest: it will create a Container App for each project resource, using the environment variables and bindings defined in the manifest.

If you want to use managed Azure services (like Azure Database for PostgreSQL), you should add them to the Bicep file and then reference them in your AppHost via `AddAzurePostgresFlexibleServer`. AZD will then provision them and inject connection strings automatically.

For our example, we'll stick with container‑based PostgreSQL (running in ACA as a container), but note that this is not recommended for production (stateful containers in ACA are possible but require persistent storage and careful handling). We'll assume you either use a managed database or you're okay with the container approach for demo purposes.

---

### 14.6 Deploying with `azd up`

Now run:

```bash
azd up
```

This command does the following:

1. **Provision** the Azure resources defined in `main.bicep` (if not already existing).
2. **Build** container images for each project (using `docker build` with the Dockerfiles we created earlier).
3. **Push** the images to Azure Container Registry.
4. **Deploy** the images to the respective Container Apps, setting environment variables, secrets, and scaling rules based on the Aspire manifest.

The process may take several minutes. You'll see output indicating each step.

Once complete, you'll get a URL for the web frontend (e.g., `https://webfrontend.xxxx.region.azurecontainerapps.io`). Open that URL to see your application running in the cloud.

---

### 14.7 How AZD Interprets the Aspire Manifest

Under the hood, AZD runs your AppHost with the manifest publisher and reads the generated JSON. It then maps each resource to a Container App:

- **Project resources** become Container Apps. The environment variables are set as environment variables in the container. Secrets are stored in the Container App's secrets and referenced.
- **Container resources** (like PostgreSQL) become Container Apps as well, running the specified image. They get environment variables and volumes if defined. However, stateful containers require persistent storage; ACA supports Azure Files volumes.
- **Bindings** become ingress settings. For example, if your web frontend has an HTTP binding, ACA will create an ingress rule with external visibility (if you want).
- **Service discovery** is handled by ACA’s internal DNS. Services can reach each other using the Container App name (e.g., `apiservice`) because they are in the same environment.

AZD also handles dependencies: it ensures that the API service starts after the database (though not strictly enforced, but you can use health checks).

---

### 14.8 Configuring Secrets and Environment Variables

In our AppHost, we used parameters for secrets. AZD will prompt for these during `azd up` (if not provided). You can also set them in the `.azure/<env>/.env` file or in Azure Key Vault (if you configure that).

For example, in the `.azure/myapp-dev/.env` file, you might have:

```
POSTGRES_PASSWORD=mysecretpassword
KEYCLOAK_CLIENT_SECRET=anothersecret
```

AZD will inject these into the Container Apps as secrets.

---

### 14.9 Scaling Rules in ACA

One of the benefits of ACA is automatic scaling. By default, your Container Apps scale to zero when idle (if you have no minimum replicas). You can configure scaling rules based on:

- **HTTP traffic**: number of concurrent requests.
- **Event‑driven** (KEDA): e.g., Azure Service Bus queue length, RabbitMQ queue length.
- **CPU/Memory**: custom rules.

In your AppHost, you can add scaling annotations to resources. For example, to tell ACA to scale based on HTTP traffic:

```csharp
var apiService = builder.AddProject<Projects.MyAspireApp_ApiService>("apiservice")
    .WithHttpScale(maxConcurrentRequests: 100);
```

AZD will translate this into a scaling rule in the Container App definition.

For the worker service, you might scale based on the RabbitMQ queue length. You can use KEDA scalers, but that requires additional configuration in the Bicep or through annotations.

---

### 14.10 Monitoring and Logs

Once deployed, you can monitor your application using Azure Monitor. AZD sets up a Log Analytics workspace automatically. You can view logs and metrics in the Azure portal:

- Go to your Container Apps environment.
- Select the individual Container App (e.g., `apiservice`).
- Under "Monitoring", you can see logs, metrics, and even live traces if you have Application Insights enabled.

Aspire’s service defaults include OpenTelemetry exporters. In ACA, you can configure the OTLP endpoint to send telemetry to Azure Monitor. You can add the Application Insights SDK or use the OpenTelemetry Azure Monitor exporter.

To enable Application Insights, you can add it in the Bicep and configure the Container App to set the `APPLICATIONINSIGHTS_CONNECTION_STRING` environment variable. Then, in your service, add the Azure Monitor exporter.

---

### 14.11 Hands‑on: Deploying the E‑Commerce App to ACA

Let's walk through deploying our e‑commerce application to Azure Container Apps.

#### Step 1: Prepare the Azure Files

- Ensure you have an `azure.yaml` as described.
- Create Dockerfiles for each project if you haven't already (as in Chapter 13).
- Make sure your AppHost can run without errors (test `dotnet run` in AppHost).

#### Step 2: Run `azd init`

```bash
azd init
```

Choose a name, subscription, region.

#### Step 3: Review and Modify Bicep (Optional)

Open the generated `main.bicep` in the `infra` folder. You may want to add:

- A PostgreSQL Flexible Server resource (if you want managed database). There are ready‑made modules for this.
- Application Insights resource.
- Adjust scaling rules.

For simplicity, we'll keep the default, which will create a Container Apps environment and one Container App per service.

#### Step 4: Set Parameters

Create a `.env` file in `.azure/<env>/` with any secrets, or let AZD prompt.

#### Step 5: Run `azd up`

```bash
azd up
```

This will provision, build, and deploy. At the end, note the URL for the web frontend. Open it in a browser. You should see your e‑commerce site.

#### Step 6: Test the Application

Try to view products (you may need to seed some data). Place an order. Check that the worker processes it. You can view logs in the Azure portal to confirm.

#### Step 7: Clean Up

When you're done, run `azd down` to delete all resources and avoid incurring costs.

---

### 14.12 Summary

In this chapter, you learned how to deploy your Aspire application to Azure Container Apps using the Azure Developer CLI. Key takeaways:

- AZD integrates with Aspire by reading the manifest and provisioning ACA resources.
- The `azure.yaml` file maps your projects to services.
- `azd up` handles the entire lifecycle: provision, build, push, deploy.
- ACA provides serverless containers with scaling, ingress, and monitoring.
- You can extend the Bicep to add managed services like Azure Database for PostgreSQL.

Deploying to the cloud is the final step in making your application production‑ready. In the next chapter, we’ll discuss **Production Readiness and Day 2 Operations**, covering topics like logging, monitoring, scaling, and secrets management in a live environment.

---

**Exercises**

1. Add an Azure Database for PostgreSQL Flexible Server to the Bicep template and modify the API to use it instead of the container database. Update the AppHost to reference the Azure resource using `AddAzurePostgresFlexibleServer`. Redeploy and test.
2. Configure Application Insights for your services by adding the OpenTelemetry Azure Monitor exporter and setting the connection string via environment variable. Verify that traces appear in Application Insights.
3. Set up scaling rules for the worker service based on the RabbitMQ queue length using KEDA. You'll need to add the KEDA scaler definition in the Bicep. (Hint: use `containerApp.addonProfiles`).
4. Implement a health check endpoint in the API and configure ACA’s health probes (liveness/readiness) in the Bicep.

In Chapter 15, we’ll dive into **Production Readiness and Day 2 Operations** to ensure your application runs smoothly in production.